## Запуск отдельных моделей

[Run inference on the Edge TPU with Python](https://coral.ai/docs/edgetpu/tflite-python/#overview)

In [2]:
import os
import pathlib
from pycoral.utils import edgetpu
from pycoral.utils import dataset
from pycoral.adapters import common
from pycoral.adapters import classify
from PIL import Image

In [3]:
print(f"Current directory: {os.getcwd()}")

Current directory: c:\prj\DLS\dls_prj


In [6]:
from pycoral.utils import edgetpu

def check_coral_tpu():
    try:
        # Получаем список доступных Edge TPU устройств
        devices = edgetpu.list_edge_tpus()
        
        if devices:
            print(f"Найдено {len(devices)} Edge TPU устройств:")
            for device in devices:
                print(f"- {device}")
            return True
        else:
            print("Edge TPU устройства не найдены")
            return False
            
    except Exception as e:
        print(f"Ошибка при проверке Edge TPU: {e}")
        return False

# Запуск проверки
check_coral_tpu()

Найдено 1 Edge TPU устройств:
- {'type': 'usb', 'path': '/sys/bus/usb/devices/2-5'}


True

In [7]:
# Specify the TensorFlow model, labels, and image
# script_dir = pathlib.Path(__file__).parent.absolute()
script_dir = pathlib.Path.cwd()
model_file = os.path.join(script_dir, 'pycoral/test_data/mobilenet_v2_1.0_224_inat_bird_quant.tflite')
label_file = os.path.join(script_dir, 'pycoral/test_data/inat_bird_labels.txt')
image_file = os.path.join(script_dir, 'pycoral/test_data/sora.jpg')

In [8]:

# Initialize the TF interpreter
interpreter = edgetpu.make_interpreter(model_file)
interpreter.allocate_tensors()

# Resize the image
size = common.input_size(interpreter)
image = Image.open(image_file).convert('RGB').resize(size, Image.LANCZOS)


In [ ]:

# Run an inference
common.set_input(interpreter, image)
interpreter.invoke()
classes = classify.get_classes(interpreter, top_k=3)

# Print the result
labels = dataset.read_label_file(label_file)
for c in classes:
  print('%s: %.5f' % (labels.get(c.id, c.id), c.score))


Porzana carolina (Sora): 0.97656
Haemorhous purpureus (Purple Finch): 0.00000
Nyctidromus albicollis (Common Pauraque): 0.00000


: 